<a href="https://colab.research.google.com/github/cyyeh/kaggle/blob/master/google-qa/google_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


### Download Kaggle Data Set for [TensorFlow 2.0 Question Answering](https://www.kaggle.com/c/tensorflow2-question-answering)


In [4]:
import os
os.environ['KAGGLE_USERNAME'] = "chihyuyeh" # username from the json file
os.environ['KAGGLE_KEY'] = "f21b340fc8082977cbf954c80ad69ae1" # key from the json file
!kaggle competitions download -c tensorflow2-question-answering

100% 4.46G/4.46G [01:33<00:00, 60.1MB/s]
100% 4.46G/4.46G [01:33<00:00, 51.4MB/s]
  0% 0.00/4.78M [00:00<?, ?B/s]
100% 4.78M/4.78M [00:00<00:00, 157MB/s]
  0% 0.00/18.2k [00:00<?, ?B/s]
100% 18.2k/18.2k [00:00<00:00, 15.9MB/s]


In [5]:
!unzip simplified-nq-train.jsonl.zip
!unzip simplified-nq-test.jsonl.zip

Archive:  simplified-nq-train.jsonl.zip
  inflating: simplified-nq-train.jsonl  
Archive:  simplified-nq-test.jsonl.zip
  inflating: simplified-nq-test.jsonl  


In [10]:
!pip install bert-for-tf2 sentencepiece

     |████████████████████████████████| 1.0MB 2.7MB/s 


## Initialize ALBERT Model

In [0]:
import bert
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

tf.compat.v1.disable_eager_execution()

class TDense(tf.keras.layers.Layer):
    def __init__(self,
                 output_size,
                 kernel_initializer=None,
                 bias_initializer="zeros",
                **kwargs):
        super().__init__(**kwargs)
        self.output_size = output_size
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
    def build(self,input_shape):
        dtype = tf.as_dtype(self.dtype or tf.keras.backend.floatx())
        if not (dtype.is_floating or dtype.is_complex):
          raise TypeError("Unable to build `TDense` layer with "
                          "non-floating point (and non-complex) "
                          "dtype %s" % (dtype,))
        input_shape = tf.TensorShape(input_shape)
        if tf.compat.dimension_value(input_shape[-1]) is None:
          raise ValueError("The last dimension of the inputs to "
                           "`TDense` should be defined. "
                           "Found `None`.")
        last_dim = tf.compat.dimension_value(input_shape[-1])
        self.input_spec = tf.keras.layers.InputSpec(min_ndim=3, axes={-1: last_dim})
        self.kernel = self.add_weight(
            "kernel",
            shape=[self.output_size,last_dim],
            initializer=self.kernel_initializer,
            dtype=self.dtype,
            trainable=True)
        self.bias = self.add_weight(
            "bias",
            shape=[self.output_size],
            initializer=self.bias_initializer,
            dtype=self.dtype,
            trainable=True)
        super(TDense, self).build(input_shape)
    def call(self,x):
        return tf.matmul(x,self.kernel,transpose_b=True)+self.bias

In [0]:
def mk_model(batch_size=16, max_seq_len=512):
    unique_id  = tf.keras.Input(shape=(1,), batch_size=batch_size, dtype=tf.int64,name='unique_id')
    input_ids   = tf.keras.Input(shape=(max_seq_len,), batch_size=batch_size, dtype=tf.int32,name='input_ids')
    input_mask  = tf.keras.Input(shape=(max_seq_len,), batch_size=batch_size, dtype=tf.int32,name='input_mask')
    segment_ids = tf.keras.Input(shape=(max_seq_len,), batch_size=batch_size, dtype=tf.int32,name='segment_ids')
    
    # import albert
    albert_module = hub.load(
        "https://tfhub.dev/google/albert_base/2")
    albert_outputs = albert_module.signatures["tokens"](input_ids=input_ids, segment_ids=segment_ids, input_mask=input_mask)

    pooled_output = albert_outputs["pooled_output"]
    sequence_output = albert_outputs["sequence_output"]
    
    logits = TDense(2,name='logits')(sequence_output)
    start_logits,end_logits = tf.split(logits,axis=-1,num_or_size_splits= 2,name='split')
    start_logits = tf.squeeze(start_logits,axis=-1,name='start_squeeze')
    end_logits   = tf.squeeze(end_logits,  axis=-1,name='end_squeeze')
    
    ans_type      = TDense(5,name='ans_type')(pooled_output)
    return tf.keras.Model([input_ for input_ in [unique_id,input_ids,input_mask,segment_ids] 
                           if input_ is not None],
                          [unique_id,start_logits,end_logits,ans_type],
                          name='bert-baseline')

In [39]:
model = mk_model()
model.summary()

/tensorflow-2.1.0/python3.6/tensorflow_core/python/client/session.py:1474: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  run_metadata_ptr)


Model: "bert-baseline"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
segment_ids (InputLayer)        [(16, 512)]          0                                            
__________________________________________________________________________________________________
input_ids (InputLayer)          [(16, 512)]          0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(16, 512)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_StatefulPartitioned [(16, 768), (16, 512 0           segment_ids[0][0]                
                                                                 input_ids[0][0]      